# Star shaped qubit

We'll be creating a 2D design and adding a star shaped qubit QComponent.


There's a central star shaped component. This component is essentially a circle with trpezoids subtracted from it. A T-junction qubit is attached to this central shape. Then there are four coupling resonators and one readout resonator present in the default setting. The shape of the central component and the number coupling resonators are customizable. Let's try out a few diferent ways to create and render this component.

In [1]:
# So, let us dive right in. For convenience, let's begin by enabling
# automatic reloading of modules when they change.
%load_ext autoreload
%autoreload 2

Next, we import the relevant modules. You can add new modules as and when needed. 

In [2]:
import qiskit_metal as metal
import numpy as np
from math import *
from qiskit_metal import designs, draw, Dict
from qiskit_metal.qlibrary.core import BaseQubit, QComponent

Next we import the GUI

In [3]:
from qiskit_metal import MetalGUI

A QDesign class must be instantiated each time a new quantum circuit design is created. The design class `DesignPlanar` is best for 2D circuit designs.

In [4]:
design = designs.DesignPlanar()
gui = MetalGUI(design)
gui.rebuild() 

# A Star Qubit

You can create a ready-made star qubit from the QComponent Library, qiskit_metal.qlibrary.qubits. star_qubit.py is the file containing our qubit so StarQubit is the module we import. 


In [5]:
from qiskit_metal.qlibrary.qubits.star_qubit import StarQubit

In [9]:
# The following default_options can be overridden by user.
StarQubit.get_template_options(design)

{'radius': '300um',
 'center_radius': '100um',
 'gap_couplers': '25um',
 'gap_readout': '10um',
 'connector_length': '75um',
 'trap_offset': '20um',
 'junc_w': '120um',
 'junc_h': '30um',
 'rotation1': '0.0',
 'rotation2': '72.0',
 'rotation3': '144.0',
 'rotation4': '216.0',
 'rotation5': '288.0',
 'pos_x': '0um',
 'pos_y': '0um',
 'number_of_connectors': '1',
 'resolution': '16',
 'cap_style': 'round',
 'subtract': 'False',
 'helper': 'False',
 'chip': 'main',
 'layer': '1'}

In [7]:
# To force overwrite a QComponent with an existing name.  
# This is useful when re-running cells in a notebook. 
design.overwrite_enabled = True

In [15]:
# Place the qubit at (x,y) =(0,0)
qubit_options = dict(
    pos_x = '5um',
    pos_y = '5um',  
    rotation1 = '36.0', # rotation for one of the coupling resonators '36.0',
    rotation2 = '108.0', # rotation for the readout resonator '108.0',
    rotation3 = '180.0', # rotation for one of the coupling resonators '180.0',
    rotation4 = '252.0', # rotation for one of the coupling resonators'252.0',
    rotation5 = '324.0', # rotation for one of the coupling resonators '324.0',
    number_of_connectors = 4, 
    junc_w='150um',  # transmon pocket width
    junc_h='30um',  # transmon pocket height
)

# Create a new Concentric Transmon object with name 'Q1' 
q1 = StarQubit(design, 'Star', options=qubit_options)

gui.rebuild()  # rebuild the design and plot 
gui.autoscale() #resize GUI to see QComponent
gui.zoom_on_components(['Star']) #Can also gui.zoom_on_components([q1.name])

In [ ]:
#StarQubit(design, 'Star')
#gui.rebuild()